**ALLER AU GPU EN PREMIER !!!**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

file_path = "/content/drive/MyDrive/Colab Notebooks/Memoire_M1/Test-format/Corona2.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Affichage des 2 premières entrées de la liste "examples" (si elle existe) afin de vérifier la bonne lecture des données
if "examples" in data and isinstance(data["examples"], list):
    print(json.dumps(data["examples"][:1], indent=4, ensure_ascii=False))
else:
    print("Format de données inattendu :", type(data))


Mounted at /content/drive
[
    {
        "id": "18c2f619-f102-452f-ab81-d26f7e283ffe",
        "content": "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]",


In [2]:
import os

# Définir le chemin du dossier de sortie
output_dir = "/content/drive/MyDrive/Colab Notebooks/Memoire_M1/Test-format/Sortie_3"

# Créer le dossier s'il n'existe pas
os.makedirs(output_dir, exist_ok=True)

print(f"Dossier de sortie créé à : {output_dir}")

Dossier de sortie créé à : /content/drive/MyDrive/Colab Notebooks/Memoire_M1/Test-format/Sortie_3


Début du tuto

In [3]:
#Installation spacy-transformers
!pip install spacy[transformers]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [4]:
import json
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)
print(data['examples'][0])

{'id': '18c2f619-f102-452f-ab81-d26f7e283ffe', 'content': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'metadata': {}, 'annotations': [{'id': '0825a1bf-

On a vérifié le format des données. Dans la cellule d'après on crée training_data qui contiendra uniquement les classes et les annotations, autrement dit le texte (content) et les annotations (start,end, label).

In [5]:
training_data = {'classes' : ['MEDICINE', "MEDICALCONDITION", "PATHOGEN"], 'annotations' : []}
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data['annotations'].append(temp_dict)

print(training_data['annotations'][0])

# Chemin du fichier training
output_path = os.path.join(output_dir, "training_data.json")

# Sauvegarde en JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(training_data, f, ensure_ascii=False, indent=4)

print(f"Fichier enregistré : {output_path}")

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679,

Remarques sur le format produit


*   dictionnaire à 2 clés principales "text" et "entities"
*   chaque entité suit le format (start, end, label)
* les labels sont tous en majuscules (pré-requis de spacy ??)
*"text" contient un paragraphe annoté
*"entities" contient plusieurs annotations correspondant aux termes médicaux extraits
Maintenant, il faut convertir ces données ci en format binaire DocBin

In [6]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [ ]:
# #Code totalement issue du tuto, on voit qu'il y a des entités skippées

# from spacy.util import filter_spans

# for training_example  in tqdm(training_data['annotations']):
#     text = training_example['text']
#     labels = training_example['entities']
#     doc = nlp.make_doc(text)
#     ents = []
#     for start, end, label in labels:
#         span = doc.char_span(start, end, label=label, alignment_mode="contract")
#         if span is None:
#             print("Skipping entity")
#         else:
#             ents.append(span)
#     filtered_ents = filter_spans(ents)
#     doc.ents = filtered_ents
#     doc_bin.add(doc)

# doc_bin.to_disk("training_data.spacy") # save the docbin object

# # Enregistrer le fichier dans le bon dossier
# output_path = os.path.join(output_dir, "training_data.spacy")
# doc_bin.to_disk(output_path)  # Sauvegarde dans le dossier de sortie

# print(f"Fichier enregistré avec succès : {output_path}")

100%|██████████| 31/31 [00:00<00:00, 478.19it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Fichier enregistré avec succès : /content/drive/MyDrive/Colab Notebooks/Memoire_M1/Test-format/Sortie_3/training_data.spacy


Pourquoi certaines entités n'ont pas été traitées:


*   l'index start et end ne correspond pas exactement aux limites d'un mot (problème d'aligmement avec certains caractères)
*   mauvais traitement des caractères spéciaux et espaces vides qui modifient les index ?



In [7]:
from spacy.util import filter_spans

# Vérifier les entités qui posent problème
for training_example in tqdm(training_data['annotations']):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []

    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="expand")  # Changer "contract" en "expand" afin de capturer l'entité e nentier au lieu de couper en plein milieu
        if span is None: # Si le problème n'est pas réslu et que certaines entités ne sont toujours pas traitées
            print(f"Skipping entity: '{text[start:end]}' ({label}) | Indices: {start}-{end}")
        else:
            ents.append(span)

    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy")
print("Enregistrement terminé")

# Enregistrer le fichier
output_path = os.path.join(output_dir, "training_data.spacy")
doc_bin.to_disk(output_path)  # Sauvegarde dans dossier sortie

print(f"Fichier enregistré avec succès : {output_path}")

100%|██████████| 31/31 [00:00<00:00, 512.95it/s]


Enregistrement terminé
Fichier enregistré avec succès : /content/drive/MyDrive/Colab Notebooks/Memoire_M1/Test-format/Sortie_3/training_data.spacy


Ensuite on génère le fichier de configuration pour spacy:


1.   en anglais (en)
2.   composants (ner uniquement ici pas comme dans le tuto car problème lors de l'entraînement avec "tagger, parser" en plus)
3.   optimisation accuracy
4.   utilisation GPU transformers

Utilisation de la commande pour ne pas avoir à se soucier d'un nouveau chemin de fichier vers config.cfg.
Première étape = générer la configuration souhaitée
Deuxième étape = compléter le fichier avec les valeurs par défaut (ne semblait pas nécessaire vu le message affiché).



In [8]:
!python -m spacy init config base_config.cfg --lang en --pipeline ner --optimize accuracy --gpu

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: GPU
- Transformer: roberta-base
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
!python -m spacy init fill-config base_config.cfg config.cfg

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Ensuite pour commencer l'entraînement je dois vérifier que GPU est disponible: je suis ici sur colab donc on doit faire : exécution > modifier le type d'exécution et choisi dans "accélérateur matériel" la case GPU (à noter que le nombre d'utilisation est limité donc mieux sur ordi mais j'ai un problème avec le mien) + TRES IMPORTANT, IL FAUT FAIRE RETOURNER LES AUTRES CELLULES DONC A FAIRE AU DEBUT DU PROGRAMME

In [10]:
import torch
print("GPU disponible :", torch.cuda.is_available())

import spacy
print("spaCy GPU support :", spacy.prefer_gpu())

!nvidia-smi

GPU disponible : True
spaCy GPU support : True
Tue Feb 11 12:29:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             25W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+

On lance donc l'entraînement avec l'option GPU !
(problème pour enregistrer à distance dans drive donc on doit tout télécharger c'est un peu long à faire ....)

In [11]:
!python -m spacy train config.cfg --output ./ --paths.train training_data.spacy --paths.dev training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 166kB/s]
config.json: 100% 481/481 [00:00<00:00, 3.66MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 4.09MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 3.14MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 16.1MB/s]
2025-02-11 12:30:13.457128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739277013.690189    1483 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739277013.760277    1483 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 12:30:14.260775: I tensorflow/

In [ ]:
# 32 minutes sur google colab

In [12]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("Antiretroviral therapy (ART) is recommended for all HIV-infected\
individuals to reduce the risk of disease progression.\nART also is recommended \
for HIV-infected individuals for the prevention of transmission of HIV.\nPatients \
starting ART should be willing and able to commit to treatment and understand the\
benefits and risks of therapy and the importance of adherence. Patients may choose\
to postpone therapy, and providers, on a case-by-case basis, may elect to defer\
therapy on the basis of clinical and/or psychosocial factors.")

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.11/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded vi